In [149]:
from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import copy
import numpy as np
import torch
from unidepth.utils import colorize, image_grid
from unidepth.models import UniDepthV1
from unidepth.utils.visualization import colorize_np
from unidepth.utils.geometric import project_points
from PIL import Image
import open3d as o3d
import cv2
import fuse_depth as fd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
model = UniDepthV1.from_pretrained("lpiccinelli/unidepth-v1-vitl14")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

base_path = "image_8.png"
H,W  = np.array(Image.open(base_path)).shape[:2]

support_paths = []
for filename in listdir("supports"): 
    full_path = join("supports", filename)
    support_paths.append(full_path)


sup_pcds = []
for i, sup_path in enumerate(support_paths):
    T, scale, torch_intrinsics1, torch_points1 = fd.pairwise_pose(base_path, sup_path, model) # base, sup order
    if scale > 1.2 or scale < 0.9:
        continue
    pcd1 = o3d.io.read_point_cloud("ply2/im1.ply")
    pcd2 = o3d.io.read_point_cloud("ply2/im2.ply")
    pcd2 = pcd2.scale(scale, center=(0, 0, 0))
    pcd2 = pcd2.transform(T)
    o3d.io.write_point_cloud(f"ply2/{i}.ply", pcd2)
    sup_pcds.append(pcd2)

Instantiate: dinov2_vitl14
0.9666967638737914
0.9745800862325769
1.0194454043313603
1.0140443181375922


In [151]:
def icp(pcd1, pcd2):
    source = copy.deepcopy(pcd1)
    source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    target = copy.deepcopy(pcd2)
    target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, 0.1, np.eye(4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
    print("Transformation is:")
    print(reg_p2p.transformation)
    return reg_p2p.transformation

In [152]:
pcd_comb = copy.deepcopy(pcd1)
for pcd in sup_pcds:
    T = icp(pcd, pcd1)
    dup_pcd = copy.deepcopy(pcd)
    dup_pcd.transform(T)
    pcd_comb += dup_pcd

Transformation is:
[[ 0.99997074  0.00322195 -0.00693759  0.04197298]
 [-0.0031981   0.99998895  0.00344604 -0.04152347]
 [ 0.00694861 -0.00342375  0.99997     0.05695842]
 [ 0.          0.          0.          1.        ]]
Transformation is:
[[ 0.99970674 -0.00802815 -0.02284698  0.16833429]
 [ 0.00836004  0.99986037  0.01446861 -0.09075355]
 [ 0.02272764 -0.01465536  0.99963427  0.08234831]
 [ 0.          0.          0.          1.        ]]
Transformation is:
[[ 9.99949587e-01  3.54623954e-03  9.39396956e-03 -9.06751107e-02]
 [-3.55210052e-03  9.99993507e-01  6.07297808e-04 -1.77446787e-02]
 [-9.39175494e-03 -6.40635516e-04  9.99955691e-01  1.71750380e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Transformation is:
[[ 0.99998818  0.00330423 -0.00356781  0.01028848]
 [-0.00326413  0.99993209  0.01118753 -0.06911063]
 [ 0.00360454 -0.01117575  0.99993105 -0.00282219]
 [ 0.          0.          0.          1.        ]]


In [169]:
depth_comb = fd.create_depth_map(np.asarray(pcd_comb.points), torch_intrinsics1.squeeze().cpu().numpy(), (H,W))
depth_comb = cv2.medianBlur(depth_comb.astype('float32'), 5)

depth_col_avg = colorize_np(depth_comb, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_col_avg).save("median.png")

points_tensor = torch.tensor(np.asarray(pcd_comb.points), dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()
torch_depth_mean = project_points(points_tensor, torch_intrinsics1, (H,W))
depth_pred_col_mean = colorize(torch_depth_mean.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col_mean).save("mean.png")

point_cloud_flat = torch_points1.view(1, 3, -1)  # Shape: (B, 3, HW)
point_cloud_transposed = point_cloud_flat.transpose(1, 2)  # Shape: (B, HW, 3)
torch_depth = project_points(point_cloud_transposed, torch_intrinsics1, (H,W))
depth_pred_col = colorize(torch_depth.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("original.png") 

In [ ]:
gt = np.load("COS429_Final/data/environment1/depths/depth_8.npy")
gt_col = colorize_np(gt, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(gt_col).save("gt.png")

In [155]:
og_depth = torch_depth.squeeze().cpu().numpy()
mean_depth = torch_depth_mean.squeeze().cpu().numpy()

og_depth[np.isnan(gt)] = 0
depth_comb[np.isnan(gt)] = 0
mean_depth[np.isnan(gt)] = 0 
gt[np.isnan(gt)] = 0


In [156]:
mse = np.mean((depth_comb - gt) ** 2)
print("Median Reconstructed Depth MSE:", mse)

mse = np.mean((mean_depth - gt) ** 2)
print("Mean Reconstructed Depth MSE:", mse)

mse = np.mean((og_depth - gt) ** 2)
print("Original Depth MSE:", mse)

Median Reconstructed Depth MSE: 6.1231437
Mean Reconstructed Depth MSE: 6.1895876
Original Depth MSE: 6.098673
